In [ ]:
from csv import DictReader

# dict de dict avec product_footprint -> 
exports_from_marseille = {}
exports_to_marseille = {}
exports_to_marseille_footprints_only = set()
values = set()
product_classification_field = 'product_RE_aggregate'
total_value_to_pf = 0
with open('../../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        value = int(row['value'].split('.')[0] or 0)
        product = row[product_classification_field]
        origin = row['origin_province'] or 'sans origine'
        product_footprint = product + ' (' + origin + ')'
        # exported from Marseille direction des fermes
        if \
                row['customs_region'] == 'Marseille' \
            and row['year'] == '1789' \
            and row['export_import'] == 'Exports' \
            and row['best_guess_region_prodxpart'] == '1' \
            :
        
            if product_footprint not in exports_from_marseille:
                exports_from_marseille[product_footprint] = 0
            exports_from_marseille[product_footprint] += value
        # exported to Marseille port franc
        if \
                row['partner_simplification'] == 'Marseille' \
            and row['year'] == '1789' \
            and row['export_import'] == 'Exports' \
            and row['best_guess_region_prodxpart'] == '1' \
            and row['customs_region'] != 'Marseille' \
            :
            total_value_to_pf += value
            exporter = row['customs_region']
            exports_to_marseille_footprints_only.add(product_footprint)
            if exporter not in exports_to_marseille:
                exports_to_marseille[exporter] = {}
            if product_footprint not in exports_to_marseille[exporter]:
                exports_to_marseille[exporter][product_footprint] = 0
            exports_to_marseille[exporter][product_footprint] += value

for_sankey = []
for product, value in exports_from_marseille.items():
    # take if the footprint is  in exports to the port
    if product in exports_to_marseille_footprints_only:
        for_sankey.append({
            "from": product,
            "to": "exports de la DF de Marseille",
            "value": value
        })
    # else:
    #    print('exclude ' + product)
for customs_region, products in exports_to_marseille.items():
    for product, value in products.items():
        # filter to what is exported by df of Marseille only
        if product in exports_from_marseille:
            for_sankey.append({
                "from": customs_region,
                "to": product,
                "value": value
            })
print('total value exports to port franc Marseille : ' + str(total_value_to_pf))

In [ ]:
from csv import DictWriter

flows = []
with open('toflit18_reexports_flows.csv', 'w', newline='') as csvfile:
    writer = DictWriter(csvfile, fieldnames=['from', 'to', 'value'])
    writer.writeheader()
    writer.writerows(for_sankey)
    

## Phase 2 vérifier si c'est crédible

In [ ]:
total_exports_vers_pf_marseille = 0
total_exports_vers_pf_marseille_from_df_marseille = 0
total_exports_from_df_marseille_origine_fr = 0
total_imports_df_marseille_from_partner_fr = 0

foreign_origins = set([
    'Hollande', 
    "États-Unis de l'Amérique", 
    'Saint-Réal', 
    'Nord', 
    'Levant et Barbarie', 
    'Suisse', 
    'sans origine', 
    'La Marche', 
    'Espagne', 
    'Flandre et autres états de l\'Empereur', 
    'Inconnu', 
    'Angleterre', 
    'Amérique', 
    'Portugal', 
    'Italie', 
    'Asie', 
    'Outre-Mers', 
    'Monde', 
    'Allemagne'
])

origins = set()
with open('../../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        value = int(row['value'].split('.')[0] or 0)
        # product = row[product_classification_field]
        origin = row['origin_province'] or 'sans origine'
        if \
                row['year'] == '1789' \
            and row['best_guess_region_prodxpart'] == '1' \
            :
            if row['export_import'] == 'Exports':
                if row['partner_simplification'] == 'Marseille' and row['customs_region'] != 'Marseille':
                       total_exports_vers_pf_marseille += value
                if row['partner_simplification'] == 'Marseille' and row['customs_region'] == 'Marseille':
                       total_exports_vers_pf_marseille_from_df_marseille += value
                if row['customs_region'] == 'Marseille' and origin not in foreign_origins:
                    origins.add(origin)
                    total_exports_from_df_marseille_origine_fr += value
            elif row['export_import'] == 'Imports':
                if row['customs_region'] == 'Marseille' and row['partner_grouping'] == 'France' and row['partner_simplification'] != 'Marseille':
                    print(row['partner_simplification'])
                    total_imports_df_marseille_from_partner_fr += value
        # product_footprint = product + ' (' + origin + ')'

print(origins)
print('total_exports_vers_pf_marseille : ' + str(total_exports_vers_pf_marseille))
print('total_exports_vers_pf_marseille_from_df_marseille : ' + str(total_exports_vers_pf_marseille_from_df_marseille))
print('total_exports_from_df_marseille_origine_fr : ' + str(total_exports_from_df_marseille_origine_fr))
print('total_imports_df_marseille_from_partner_fr : ' + str(total_imports_df_marseille_from_partner_fr))

overview_data = [
{
    "metrique": "Exports vers le PF de Marseille hors customs_region=Marseille (1789)",
    "value": total_exports_vers_pf_marseille
},
{
    "metrique": "Exports vers le PF de Marseille depuis la DF de Marseille - customs_region=Marseille (1789)",
    "value": total_exports_vers_pf_marseille_from_df_marseille,
},
{
    "metrique": "Exports depuis la DF de Marseille pour des produits d'origine française (1789)",
    "value": total_exports_from_df_marseille_origine_fr
},
{
    "metrique": "Imports de la DF de Marseille depuis des partenaires français",
    "value": total_imports_df_marseille_from_partner_fr
}
]

In [ ]:
from IPython.display import display
import pandas as pd


def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)
    
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Comparaison des nombres pour Marseille",
    "width": 800,
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "bar"
    },
    "data": {
        "values": overview_data
    },
    "encoding": {
        "x": {
            "field": "value",
            "type": "quantitative", 
            "title": "valeur en lt",
            "axis": {
                "orient": "top",            }
        },git status
        
        "y": {
            "field": "metrique",
            "type": "nominal",
            "title": "",
            "axis": {
                "labelLimit": 500,
            }
        }
    }
})